In [6]:
import importlib
import frengression
# importlib.reload(frengression)
from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')
from CausalEGM import *

import numpy as np
import jax.numpy as jnp
import pickle
import os
from tqdm import tqdm
import src.exp_utils as exp_utils

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import copy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss
from scipy.spatial.distance import pdist, squareform
from scipy.stats import norm, gaussian_kde
import warnings

warnings.filterwarnings("ignore")


In [14]:
s,x,z,y = generate_data_survivl(n=10,T=5, random_seed=42, C_coeff=0)
s_tr = torch.tensor(s, dtype=torch.float32)
x_tr = torch.tensor(x, dtype=torch.int32)
y_tr = torch.tensor(y, dtype=torch.int32)
z_tr = torch.nan_to_num(torch.tensor(z, dtype=torch.float32))

model = FrengressionSeq(x_dim=1, y_dim=1, z_dim=1, T=5, s_dim = 1, noise_dim=1, num_layer=3, hidden_dim=100, 
                        device=device, x_binary = True, s_in_predict=True, y_binary=True)
# model.train_y(s=s_tr,x=x_tr,z=z_tr,y=y_tr,num_iters=4000, lr=1e-4, print_every_iter=1000)

In [8]:
event_indicator = (y_tr>0).float()
c = torch.cumsum(event_indicator, dim=1)
c_shifted = torch.zeros_like(c)
c_shifted[:, 1:] = c[:, :-1]
mask = (c_shifted > 0)
y_masked = copy.deepcopy(y_tr)
y_masked[mask] = -1

In [9]:
y_tr

tensor([[1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0]], dtype=torch.int32)

In [11]:
z_tr

tensor([[-0.3250,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8188, -0.0544, -0.5053, -0.8043,  0.0000],
        [-0.5669,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.1772,  0.0681,  0.7861,  1.3642,  0.4514],
        [-0.2943, -1.4967,  0.0000,  0.0000,  0.0000],
        [ 0.6727,  0.5753, -0.7639,  0.7046,  0.0000],
        [ 1.3669, -0.7025, -1.2175,  0.0000,  0.0000],
        [-1.4821, -0.1779,  0.0000,  0.0000,  0.0000],
        [-0.4471,  0.7484,  0.0560, -0.1865, -0.5097],
        [-0.3443,  0.9819,  0.0000,  0.0000,  0.0000]])

In [10]:
y_masked

tensor([[ 1, -1, -1, -1, -1],
        [ 0,  0,  0,  1, -1],
        [ 1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0],
        [ 0,  1, -1, -1, -1],
        [ 0,  0,  0,  1, -1],
        [ 0,  0,  1, -1, -1],
        [ 0,  1, -1, -1, -1],
        [ 0,  0,  0,  0,  1],
        [ 0,  1, -1, -1, -1]], dtype=torch.int32)

In [20]:
mask

tensor([[False,  True,  True,  True,  True],
        [False, False, False, False,  True],
        [False,  True,  True,  True,  True],
        [False, False, False, False, False],
        [False, False,  True,  True,  True],
        [False, False, False, False,  True],
        [False, False, False,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False, False, False],
        [False, False,  True,  True,  True]])

In [21]:
eta1 = model.sample_eta(s=s_tr, x=x_tr,z=z_tr)
y_sample1 = model.sample_y(s=s_tr,x=x_tr, eta = eta1)

eta2 = model.sample_eta(s=s_tr, x=x_tr,z=z_tr)
y_sample2 = model.sample_y(s=s_tr,x=x_tr, eta = eta1)

In [22]:
eta1[mask]=-1

In [23]:
eta1

tensor([[-0.0112, -1.0000, -1.0000, -1.0000, -1.0000],
        [ 0.0216, -0.1930,  0.0050, -0.0046, -1.0000],
        [-0.0590, -1.0000, -1.0000, -1.0000, -1.0000],
        [-0.0311, -0.1392,  0.0320, -0.0242,  0.0786],
        [-0.0659, -0.2234, -1.0000, -1.0000, -1.0000],
        [ 0.0207, -0.1456,  0.0276, -0.0498, -1.0000],
        [ 0.0193, -0.1261,  0.0656, -1.0000, -1.0000],
        [-0.1036, -0.1336, -1.0000, -1.0000, -1.0000],
        [-0.0081, -0.2795,  0.0027, -0.0183,  0.0376],
        [ 0.0081, -0.1945, -1.0000, -1.0000, -1.0000]],
       grad_fn=<IndexPutBackward0>)

In [5]:
y_sample1[mask]=-1

NameError: name 'mask' is not defined

In [14]:
from torch.linalg import vector_norm

In [24]:
vector_norm(y_sample1 -y_tr, 2, dim=1)

tensor([9.9729e-01, 1.2761e-04, 4.4359e-03, 2.1259e-04, 8.1114e-02, 4.7928e-04,
        1.4730e-04, 6.1597e-04, 4.6929e-01, 1.7108e-04],
       grad_fn=<LinalgVectorNormBackward0>)

In [23]:
(y_sample1 -y_tr).shape

torch.Size([10, 5])

In [21]:
y_tr.shape

torch.Size([10, 5])